<h1 style="font-size: 50px;">Research Project 2 Epilepsy Ionic Modulation - SmallBrain Hippocampus Layered</h1>

PLACEHOLDER FOR SMALL EXPLANATION

<br></br><br></br>

<h2 style="font-size: 40px;">Installing and Importing Libraries</h2>

In [1]:
# Installing all the required libraries.
!pip install -q jupyter
!pip install -q matplotlib
!pip install -q numpy
!pip install -q pandas
!pip install -q plotly
!pip install -q scipy
!pip install -q tqdm
!pip install -q Brian2

In [10]:
# Importing all the required libraries.
import os
from brian2 import *
import random
from tqdm.notebook import tqdm
import numpy as np

import warnings
warnings.filterwarnings('ignore')

from plots import *
from equations import *
from global_settings import *
from masks import *
from helper import *
from run_loop import *

import time

<br></br><br></br>

<h2 style="font-size: 40px;">General Functions</h2>

In [3]:
# Reads the stimulus input signal.
def read_input_signal(file_name):
    in_1 = np.loadtxt('./stimuli/'+file_name)

    # Converting the signal to be of type 'TimedArray' with a specified time step which can be used in the simulation.
    input_signal = TimedArray(in_1*namp,dt=defaultclock.dt)

    return input_signal

<br></br>

In [12]:
# Calculating the middle y-coordinate of the stimulus as well as its radius.
def calculate_spherical_stimulus_middle_y_point_and_radius(layer_data, list_of_layers):

    # Initializing a cumulative y-coordinate that represents the height of all the layers.
    current_total_y = 0

    # Initializing a cumulative y-coordinate that represents the height of the layers only in the 'list_of_layers'.
    current_layers_y = 0

    # Initializing the variables 'y_middle_point' and 'radius_of_stimulus' that will eventually respectively contain the middle y-coordinate of the current layer as well as a radius that makes sure only the neurons from the current layer are part of the stimulus mask.
    y_middle_point = 0
    radius_of_stimulus = 0

    # Looping over every layer and if the current layer is the layer that we are looking for, the 'current_y' variable can be used to retrieve the middle y-coordinate of the current layer. Else, add the layer height to the 'current_y' variable.
    for layer in layer_data:
        if len(list_of_layers) == 1: 
            layer_name = list_of_layers[0]
            if layer_data[layer]['layer_name'] == layer_name:
                y_middle_point = current_total_y + (layer_data[layer]['height'] / 2)
                radius_of_stimulus = (layer_data[layer]['height'] / 2) * 0.99  # This is multiplied by 0.99 to ensure that we stay within the bounds of the current layer.
            else:
                current_total_y += layer_data[layer]['height']
        
        else:
            if layer_data[layer]['layer_name'] in list_of_layers:
                if layer_data[layer]['layer_name'] == list_of_layers[-1]:
                    y_middle_point = current_total_y + ((current_layers_y + layer_data[layer]['height']) / 2)
                    radius_of_stimulus = ((current_layers_y + layer_data[layer]['height']) / 2) * 0.99  # This is multiplied by 0.99 to ensure that we stay within the bounds of the set of layers.
                else:
                    current_layers_y += layer_data[layer]['height']
            else:
                current_total_y += layer_data[layer]['height']

    return [y_middle_point, radius_of_stimulus]

<br></br><br></br>

<h2 style="font-size: 40px;">Topology and Neuron Groups Functions</h2>

In [14]:
import random

# Creating the complete neuron topology of the model taking into account the layered structure of the hippocampus.
def create_complete_neuron_topology_hippocampus_layered(total_num_of_neurons, layer_dimensions, layer_neuron_densities, layer_excitatory_ratios, layer_names):
    
    # Calculating the neuron density of the whole random topology model.
    random_x_bound, random_y_bound, random_z_bound = random_topo_bounds 
    random_topo_volume = random_x_bound * random_y_bound * random_z_bound
    random_topo_density = total_num_of_neurons / random_topo_volume

    # Calculating the sum of desired neuron densities across all the hippocampal layers.
    total_desired_density = sum(list(layer_neuron_densities.values()))

    # Calculating the total number of neurons for each layer based on the proportion of their density to the total desired density.
    neuron_counts = {layer: (density / total_desired_density) * total_num_of_neurons for layer, density in layer_neuron_densities.items()}
    
    # Initializing a dictionary to store information about the layers.
    layer_data = {}

    # Looping over each layer and calculating its volume and neuron count.
    for layer_name in layer_names:
        layer_density = layer_neuron_densities[layer_name]
        
        # Total neurons in the current layer
        layer_total_neurons = neuron_counts[layer_name]

        # Calculating the volume of the current layer.
        layer_volume = layer_total_neurons / random_topo_density

        # Calculating the total number of excitatory and inhibitory neurons in the current layer.
        layer_num_excitatory_neurons = layer_excitatory_ratios[layer_name] * layer_total_neurons
        layer_num_inhibitory_neurons = (1 - layer_excitatory_ratios[layer_name]) * layer_total_neurons

        # Fixing the width and the depth of the neuron layer such that only the height of the layers varies.
        layer_width = random_x_bound
        layer_depth = random_z_bound

        # Calculating the height of the layer based on the volume and the width and depth of the layer.
        layer_height = layer_volume / (layer_width * layer_depth)

        # Adding the layer data to the 'layer_data' dictionary.
        layer_data[layer_name] = {
            "layer_name": layer_name,
            "volume": layer_volume,
            "width": layer_width,
            "depth": layer_depth,
            "height": layer_height,
            "num_excitatory_neurons": int(layer_num_excitatory_neurons),
            "num_inhibitory_neurons": int(layer_num_inhibitory_neurons)         
        }

    # Initializing a cumulative y-coordinate that will be used for generating the different points across the layers.
    current_y = 0

    # Initializing two dictionaries that will for each layer store the positions of the excitatory and inhibitory neurons.
    excitatory_positions = {}
    inhibitory_positions = {}

    # Looping over every layer and randomly generating a set of excitatory and inhibitory neurons according to the neurons defined in the 'layer_data'.
    for layer in layer_data:
        layer_all_info = layer_data[layer]

        # Retrieving the number of excitatory and inhibitory neurons that should be present within the current layer.
        num_excitatory_neurons = layer_all_info['num_excitatory_neurons']
        num_inhibitory_neurons = layer_all_info['num_inhibitory_neurons']

        # Generating a set of random x, y, and z positions that fall within the current layer for the excitatory neurons.
        layer_x_positions_exc = [random.uniform(0, layer_all_info['width']) for _ in range(num_excitatory_neurons)]
        layer_y_positions_exc = [(current_y + random.uniform(0, layer_all_info['height'])) for _ in range(num_excitatory_neurons)]
        layer_z_positions_exc = [random.uniform(0, layer_all_info['depth']) for _ in range(num_excitatory_neurons)]
        excitatory_topology_layer = np.array([layer_x_positions_exc, layer_y_positions_exc, layer_z_positions_exc])

        # Generating a set of random x, y, and z positions that fall within the current layer for the inhibitory neurons.
        layer_x_positions_inh = [random.uniform(0, layer_all_info['width']) for _ in range(num_inhibitory_neurons)]
        layer_y_positions_inh = [(current_y + random.uniform(0, layer_all_info['height'])) for _ in range(num_inhibitory_neurons)]
        layer_z_positions_inh = [random.uniform(0, layer_all_info['depth']) for _ in range(num_inhibitory_neurons)]
        inhibitory_topology_layer = np.array([layer_x_positions_inh, layer_y_positions_inh, layer_z_positions_inh])

        # Updating the cumulative y-coordinate.
        current_y += layer_all_info['height']

        # Adding the 'excitatory_topology_layer' and the 'inhibitory_topology_layer' to the dictionaries 'excitatory_positions' and 'inhibitory_positions'.
        excitatory_positions[layer_all_info['layer_name']] = excitatory_topology_layer
        inhibitory_positions[layer_all_info['layer_name']] = inhibitory_topology_layer
        
    return [layer_data, excitatory_positions, inhibitory_positions]

####################################
########  TESTING FUNCTION  ########
####################################
total_num_of_neurons = 17000
layer_dimensions = {"EC": {"width": , "height": , "depth": }, 
                    "DG": {"width": , "height": , "depth": },  
                    "CA1": {"width": , "height": , "depth": }, 
                    "CA3": {"width": , "height": , "depth": }}
layer_neuron_densities = {"EC": 27400, 
                          "DG": 87100, 
                          "CA1": 7900, 
                          "CA3": 15100}
layer_excitatory_ratios = {"EC": 0.8, 
                           "DG": 0.8, 
                           "CA1": 0.8, 
                           "CA3": 0.8}
layer_names = ['EC', 'DG', 'CA1', 'CA3']

layer_data, excitatory_positions, inhibitory_positions = create_complete_neuron_topology_hippocampus_layered(total_num_of_neurons, layer_dimensions, layer_neuron_densities, layer_excitatory_ratios, layer_names)

# Calculating the total number of neurons and the total density for verification.
total_num_of_neurons_ver = 0
total_volume = 0
for layer in layer_names:
    total_num_of_neurons_ver += (len(excitatory_positions[layer][0]) + len(inhibitory_positions[layer][0]))
    total_volume += layer_data[layer]['volume']

print(f"Total DESIRED number of neurons: {total_num_of_neurons}")
print(f"Total ACTUAL number of neurons: {total_num_of_neurons_ver}")
print(f"Total DESIRED neuron density: {total_num_of_neurons / (random_topo_bounds[0] * random_topo_bounds[1] * random_topo_bounds[2])}")
print(f"Total ACTUAL neuron density: {total_num_of_neurons_ver / total_volume}")
print(layer_data)
print(excitatory_positions)

Total DESIRED number of neurons: 17000
Total ACTUAL number of neurons: 16993
Total DESIRED neuron density: 78703.70370370371
Total ACTUAL neuron density: 78671.2962962963
{'EC': {'layer_name': 'EC', 'volume': 0.037222641509433954, 'width': 0.6, 'depth': 0.6, 'height': 0.10339622641509431, 'num_excitatory_neurons': 2343, 'num_inhibitory_neurons': 585}, 'DG': {'layer_name': 'DG', 'volume': 0.11832452830188678, 'width': 0.6, 'depth': 0.6, 'height': 0.32867924528301884, 'num_excitatory_neurons': 7450, 'num_inhibitory_neurons': 1862}, 'CA1': {'layer_name': 'CA1', 'volume': 0.010732075471698114, 'width': 0.6, 'depth': 0.6, 'height': 0.029811320754716982, 'num_excitatory_neurons': 675, 'num_inhibitory_neurons': 168}, 'CA3': {'layer_name': 'CA3', 'volume': 0.02051320754716981, 'width': 0.6, 'depth': 0.6, 'height': 0.0569811320754717, 'num_excitatory_neurons': 1291, 'num_inhibitory_neurons': 322}, 'Sub': {'layer_name': 'Sub', 'volume': 0.029207547169811318, 'width': 0.6, 'depth': 0.6, 'height':

<br></br>

In [6]:
# Creates the group of excitatory neurons.
def create_group_py(topology, noise, masks, group_name='exc_group', integ_method='exponential_euler'):

    # Extracting the passed on parameters.
    x, y, z = topology
    mu_noise, sigma_noise = noise
    stimulus_mask, treatment_mask = masks

    # Initializing a group of excitatory neurons which contains as many neurons as there are total neurons in the 3D space. ????
    # It has the following parameters:
    # - py_eqs => Differential equations that define the behavior of the excitatory neuron.
    # - threshold => Neurons fire an action potential when their membrane potential 'v' exceeds the threshold 'V_th'.
    # - reset => Resets neuron states after they fire according to 'reset_eqs'.
    # - refractory => Sets a refractory period during which an excitatory neuron cannot fire again.
    # - method => Specifies the integration method for solving the differential equations.
    G_exc = NeuronGroup(len(x),py_eqs,threshold='v>V_th',reset=reset_eqs,refractory=3*ms,name=group_name, method=integ_method)

    # Initializing the membrane potential 'v' to be a random value between -60 mV and -100 mV.
    G_exc.v = '-60*mvolt-rand()*40*mvolt'

    # Sets the neurotransmitter to be used to be glutamate (which is an excitatory neurotransmitter).
    G_exc.glu = 1

    # Assigns the positions of the neurons in micrometers.
    G_exc.x = x * um
    G_exc.y = y * um
    G_exc.z = z * um

    # Sets the size of the neurons to 'taille_exc_normale'.
    G_exc.taille = taille_exc_normale

    # Sets the mean and standard deviation of the noise affecting the neurons.
    G_exc.mu_noise = mu_noise
    G_exc.sigma_noise = sigma_noise

    # Applies the treatment and the stimulus masks to the neuron.
    G_exc.treatment_mask = treatment_mask
    G_exc.stimulus_mask = stimulus_mask

    return G_exc

<br></br>

In [7]:
# Creates the group of inhibitory neurons.
def create_group_inh(topology, noise, treatment_mask, group_name='inh_group', integ_method='exponential_euler'):

    # Extracting the passed on parameters.
    x, y, z = topology
    mu_noise, sigma_noise = noise

    # Initializing a group of inhibitory neurons which contains as many neurons as there are total neurons in the 3D space.
    # It has the following parameters:
    # - inh_eqs => Differential equations that define the behavior of the inhibitory neuron.
    # - threshold => Neurons fire an action potential when their membrane potential 'v' exceeds the threshold 'V_th'.
    # - refractory => Sets a refractory period during which an inhibitory neuron cannot fire again.
    # - method => Specifies the integration method for solving the differential equations.
    G_inh = NeuronGroup(len(x),inh_eqs,threshold='v>V_th', name=group_name, refractory=3*ms,method=integ_method)

    # Initializing the membrane potential 'v' to be a random value between -60 mV and -70 mV.
    G_inh.v = -60*mvolt-rand()*10*mvolt

    # Sets the size of the neurons to 'taille_exc_normale'.
    G_inh.taille = taille_inh_normale

    # Assigns the positions of the neurons in micrometers.
    G_inh.x = x * um
    G_inh.y = y * um
    G_inh.z = z * um

    # Sets the mean and standard deviation of the noise affecting the neurons.
    G_inh.mu_noise = mu_noise
    G_inh.sigma_noise = sigma_noise

    # Applies the treatment mask to the neuron.
    G_inh.treatment_mask = treatment_mask

    return G_inh

<br></br>

In [8]:
# Creates a group for Local Field Potential (LFP) recording.
def create_group_lfp(bounds):

    # Extracting the passed on parameters.
    x_bound, y_bound, z_bound = bounds
    
    # Setting up a singular LFP electrode
    Ne = 1

    # Setting the resistivity of the extracellular field to 0.3 siemens per meter, which is within the typical range for biological tissue (0.3-0.4 S/m).
    sigma = 0.3*siemens/meter

    # Initializes a group of neurons, which will only consist of 1 representing a single LFP electrode which has three state variables:
    # - v : volt => Represents the voltage (LFP signal).
    # - x : meter => Represent the x-coordinate of the electrode.
    # - y : meter => Represent the y-coordinate of the electrode.
    # - z : meter => Represent the z-coordinate of the electrode.
    lfp = NeuronGroup(Ne, model='''v : volt
                                   x : meter
                                   y : meter
                                   z : meter''')

    # Setting the x, y, and z coordinates of the LFP electrode to the center of the bounds provided. This places the electrode at the midpoint of the defined 3D space in each dimension.
    lfp.x = x_bound/2*um
    lfp.y = y_bound/2*um
    lfp.z = z_bound/2*um

    return lfp

<br></br><br></br>

<h2 style="font-size: 40px;">Network Configuration Function</h2>

In [9]:
# Prepares the network of neurons. The topology is already there, but the neurons still need to be connected, which is done here.
def prepare_network(topologies, stimulus_mask_exc, treatment_masks, current_variables):

    # Extracting the passed on parameters, including the following parameters:
    # - p_e2e => Probability of an excitatory to excitatory neuron (synapse) connection.
    # - p_e2i => Probability of an excitatory to inhibitory neuron (synapse) connection.
    # - p_i2e => Probability of an inhibitory to excitatory neuron (synapse) connection.
    # - p_i2i => Probability of an inhibitory to inhibitory neuron (synapse) connection.
    p_e2e, p_e2i, p_i2e, p_i2i = current_variables['p']
    topology_exc, topology_inh = topologies
    treatment_mask_exc, treatment_mask_inh = treatment_masks

    
    ##########################################
    ########  Creating Neuron Groups  ########
    ##########################################
    # Creating excitatory and inhibitory neuron groups.
    G_exc = create_group_py(topology_exc, current_variables['noise_exc'], [stimulus_mask_exc, treatment_mask_exc])
    G_inh = create_group_inh(topology_inh, current_variables['noise_inh'], treatment_mask_inh)

    # Setting up a Local Field Potential (LFP) electrode.
    G_lfp = create_group_lfp(current_variables['bounds'])
    neuron_groups = [G_exc, G_inh, G_lfp]

    
    #####################################
    ########  Creating Monitors  ########
    #####################################
    # Setting up monitors that monitor the population firing rate of excitatory and inhibitory neurons.
    popmon_exc = PopulationRateMonitor(G_exc)
    popmon_inh = PopulationRateMonitor(G_inh)

    # Setting up a monitor that monitors the voltage of the LFP group.
    Mlfp = StateMonitor(G_lfp, 'v', record=True)

    # Setting up monitors that monitor the membrane potential, the noise current (and the stimulus current) for the selected excitatory/inhibitory neurons.
    statemon_exc = StateMonitor(G_exc, ('v', 'I_stim', 'I_noise'), record=[1,2,3,4,5,6], dt=0.001*second)
    statemon_inh = StateMonitor(G_inh, ('v', 'I_noise'), record=[1,2,3,4,5,6], dt=0.001*second)
    monitors = [popmon_exc, popmon_inh, statemon_exc, statemon_inh, Mlfp]


    ############################################
    ########  Connecting Neuron Groups  ########
    ############################################
    # Configuring synaptic connections between neuron groups where in the function call 'Synapses()' the first parameter is the pre-synaptic group and the second parameter is the post-synaptic group. 
    # The third parameter 'on_pre' specifies the action to be taken when a pre-synaptic neuron fires where the synaptic event will increase the post-synaptic event by a certain amount defined by:
    # - gain => A scaling factor for the synaptic strength.
    # - g_max/siemens => The maximum conductance for the type of synapse normalized by dividing by 'siemens' which is a unit of conductance.
    # - glu_pre => The amount of glutamate released from the pre-synaptic neuron upon firing.
    # MIND: This does not create synapses but instead only specifies their dynamics. The actual synapse connections are created by using the function 'connect()'.
    S_e2e = Synapses(G_exc, G_exc, on_pre="he_post+="+str(gain)+"*"+str(g_max_e/siemens)+"*siemens*glu_pre")
    S_e2i = Synapses(G_exc, G_inh, on_pre="he_post+="+str(gain)+"*"+str(g_max_e/siemens)+"*siemens*glu_pre", name='synapses_e2i')
    S_i2e = Synapses(G_inh, G_exc, on_pre="hi_post+="+str(gain)+"*"+str(g_max_i/siemens)+"*siemens", name='synapses_i2e')

    # Generating the connections between the groups of neurons which is a probabilistic process indicating that not all neurons are connected with each other but instead it is decided based on the provided probability.
    # This is done in the following way:
    # - p_e2e => Base probability of connection between neurons of particular groups. 
    # - distance^2 => Squared Euclidean distance between the pre-synaptic and post-synaptic neurons that decreases the overall probability when it is larger.
    # - umeter^2 => Controls the steepness of the exponential decay of the probability.
    S_e2e.connect(p=f'{p_e2e}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({489.9}*umeter)**2)))')
    S_e2i.connect(p=f'{p_e2i}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({489.9}*umeter)**2)))')
    S_i2e.connect(p=f'{p_i2e}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({215}*umeter)**2)))')

    # Configuring synaptic connections between the excitatory neurons and the LFP electrode by also specifies a model used for LFP calculation.
    S_lfp = Synapses(G_exc, G_lfp, model='''w : ohm*meter**2 (constant) # Weight in the LFP calculation
                                       v_post = w*((0.0*amp/meter**2)-Im_pre) : volt (summed)''')

    # Ensuring LFP voltage is updated after neuron groups.
    S_lfp.summed_updaters['v_post'].when = 'after_groups' 

    # Generating the connections between the excitatory neurons and the LFP electrode. Here, all of them will be connected since there is no probability defined.
    S_lfp.connect()

    # Setting the weight for LFP calculation which is scaled by the Euclidean distance measure.
    S_lfp.w = '(29e3 * umetre ** 2)/(4*pi*sigma)/((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2)**.5'

    synapses = [S_e2e, S_e2i, S_i2e, S_lfp]


    ############################################
    ########  Final Network Definition  ########
    ############################################
    net = Network(neuron_groups, synapses, monitors)

    return net, synapses, monitors

<br></br><br></br>

<h2 style="font-size: 40px;">Simulation Functions</h2>

In [10]:
# Runs the simulation of the neural network in discrete time fragments which includes a mechanism to alter neuron parameters based on a firing rate threshold, which simulates a treatment effect.
def run_granular_simulation(net, variables, treatment_settings, monitors):
    
    print('#######################')
    print('# Starting Simulation #')
    print('#######################')
    print()

    # Extracting the passed on parameters.
    total_duration = variables['duration']
    input_signal = read_input_signal(variables['input_signal_file'])
    time_fragment, firing_rate_threshold = treatment_settings
    popmon_exc, popmon_inh, statemon_exc, statemon_inh, Mlfp = monitors

    
    # Setting the potassium equilibrium potentials for both the excitatory and inhibitory neurons.
    Eke = variables['Eke_baseline']
    Eki = variables['Eki_baseline']
    Eke_baseline = variables['Eke_baseline']
    Eki_baseline = variables['Eki_baseline']
    
    print('Treatment Parameters:', 'time sensitivity', time_fragment, 'FR Threshold:', firing_rate_threshold)
    print()

    # Converting the time fragments to milliseconds and determining the number of batches.
    time_fragment_ms = int(time_fragment/ms)
    num_batches = int(total_duration / time_fragment)

    # For every batch, we run the simulation on the network 'net'. Here the 'tqdm()' function is used which creates a progress bar.
    for i in tqdm(range(num_batches), desc="Running Simulation"): 
        net.run(time_fragment)

        # If after running a time fragment, the average firing rate of the excitatory neurons over the last time fragment exceeds the threshold, treatment is initiated.
        # This statement refers to the scenario where epilepsy is detected and treated by adjusting the potassium equilibrium potentials of the excitatory and inhibitory neurons.
        if np.mean(popmon_exc.rate[-time_fragment_ms:]) > firing_rate_threshold:
            Eke = variables['Eke_treatment']
            Eki = variables['Eki_treatment']

<br></br>

In [14]:

# Also change this such that we put the stimulus mask in other places in a singular layer (or multiple layers) as well. Use some kind of scaling where 1 is on the edge and 0 in the center (do this for all three dimensions).

def create_stimulus_mask(shape_mask, list_of_layers, layer_names):

    # Checking if the 'list_of_layers' is empty.
    if len(list_of_layers) == 0:
        raise ValueError("The 'list_of_layers' list is empty.")
    
    # Checking if the 'list_of_layers' only contains valid layer names.
    if not all(layer in layer_names for layer in list_of_layers):
        raise ValueError("The 'list_of_layers' list contains an invalid layer name.")
    
    # Checking if there are no duplicate layer in the 'list_of_layers'.
    if len(list_of_layers) != len(set(list_of_layers)):
        raise ValueError("The 'list_of_layers' list contains duplicate layers.")

    # Checking that the 'list_of_layers' list only contains layers that are adjacent to each other (otherwise the stimulus mask would pass through layers that are not in this list).
    if len(list_of_layers) > 1:
        layer_indices = [layer_names.index(layer) for layer in list_of_layers]
        if not all(layer_indices[i] + 1 == layer_indices[i+1] for i in range(len(layer_indices) - 1)):
            raise ValueError("The 'list_of_layers' list contains layers that are not adjacent to each other.")

    
    if shape_mask == "spherical":
        # Setting the middle point of the stimulus mask.
        stimulus_layer = list_of_layers[0]
        x_middle_point = layer_data[stimulus_layer]['width'] / 2
        z_middle_point = layer_data[stimulus_layer]['depth'] / 2
        y_middle_point, radius_of_stimulus = calculate_stimulus_middle_y_point_and_radius(layer_data, list_of_layers)
        coord_of_stimulus = [[x_middle_point, y_middle_point, z_middle_point]]
    
        return [coord_of_stimulus, radius_of_stimulus]


    elif shape_mask == "cubical":
        if len(list_of_layers) == 1:


        else:
            

    
    else:
        raise ValueError("The shape of the mask given is not valid. Please select either 'spherical' or 'cubical'.")


In [16]:
layer_names = ['EC', 'DG', 'CA1', 'CA3', 'Sub']
create_stimulus_mask('spherical', ['EC'], layer_names)

<br></br>

In [11]:
# This function manages the overall process of setting up and running multiple instances of a neural network simulation as after setting everything up, the function 'run_granular_simulation()' is called.
# In addition, it also handles the storing of the results.
def run_model_loop(variables):

    # Checking whether lengths of each variable list is equal.
    if not check_dict_lenghts(variables):
        raise ValueError('Lenghts of each variable list has be to equal!')

    
    ############################################
    ########  Setting up Stimulus Mask  ########
    ############################################
    # Setting which layer(s) will feature the stimulus mask and what kind of stimulus mask should be constructed.
    stimulus_layers = ['EC']
    shape_mask = 'spherical'

    # Creating the stimulus mask by calling the function 'create_stimulus_mask()'.
    coord_of_stimulus, radius_of_stimulus = create_stimulus_mask(shape_mask, stimulus_layers, variables['layer_names'])
    
    # Setting the overall geometry settings of the stimulus mask which includes both the middle point as well as the radius.
    stimulus_geometry_settings = [coord_of_stimulus, radius_of_stimulus]

    # Updating the 'variables'.
    variables['coord_of_stimulus'] = coord_of_stimulus * variables['copy_times']
    variables['radius_of_stimulus'] = radius_of_stimulus * variables['copy_times']
    
    # HERE SET SOME VARIABLES FOR THE ELECTRODE CALCULATION
    # distance between masks = 1/6 * lowest dimension
    # radius of electrode = 1/3 * lowest dimension


    ############################################
    ########  Setting up Electrode  ############
    ############################################
    # Adding the coordinates of the electrode to the variables array.
    variables['coord_of_electrode'] = populate_electrode_positions(variables)

    
    # For every run provided in the 'run_id' field of the variables dictionary we perform the simulation (which requires some setting up first).
    for i in range(len(variables['run_id'])):

        ##########################################
        ########  Basic Simulation Setup  ########
        ##########################################
        # Resetting the state of the simulation environment (to avoid interference from the previous run) and seeting the default simulation time step.
        start_scope()
        defaultclock.dt = 0.001*second   

        # Extracting the list of variables required for the current run.
        current_variables = {key: variables[key][i] for key in variables}
        print(current_variables['noise_exc'])

        # Retrieving the treatment settings.
        treatment_settings = [current_variables['device_sensitivity'], current_variables['firing_rate_threshold']]

        # Creating a folder to store the results.
        run_id = current_variables['run_id']
        os.mkdir(f'./results/{run_id}')
        write_run_settings(current_variables, run_id)

        
        #######################################
        ########  Creating Topologies  ########
        #######################################
        # Creating the topologies of both the excitatory and inhibitory neurons.
        layer_data, excitatory_positions, inhibitory_positions = create_complete_neuron_topology_hippocampus_layered(current_variables['N'], current_variables['random_topo_bounds'], current_variables['layer_densities'], current_variables['excitatory_ratios'], current_variables['layer_names'])

        # Assigning the topologies of the excitatory neurons of the different layers to separate variables.
        topology_EC_exc = excitatory_positions['EC']
        topology_DG_exc = excitatory_positions['DG']
        topology_CA1_exc = excitatory_positions['CA1']
        topology_CA3_exc = excitatory_positions['CA3']
        topology_Sub_exc = excitatory_positions['Sub']

        # Assigning the topologies of the inhibitory neurons of the different layers to separate variables.
        topology_EC_inh = inhibitory_positions['EC']
        topology_DG_inh = inhibitory_positions['DG']
        topology_CA1_inh = inhibitory_positions['CA1']
        topology_CA3_inh = inhibitory_positions['CA3']
        topology_Sub_inh = inhibitory_positions['Sub']

        # Adding all the topologies of the neurons to a list.
        topologies_exc = [topology_EC_exc, topology_DG_exc, topology_CA1_exc, topology_CA3_exc, topology_Sub_exc]
        topologies_inh = [topology_EC_inh, topology_DG_inh, topology_CA1_inh, topology_CA3_inh, topology_Sub_inh]

        
        ##########################################
        ########  Creating Stimulus Mask  ########
        ##########################################
        # Creating the stimulus mask. --> MAYBE loop over them all here and simply add all the neurons of a certain layer if it is in the list of layers.
        topology_exc = 0
        for stimulus_layer in stimulus_layers:
            match stimulus_layer:
                case 'EC':
                    topology_exc = topology_EC_exc
                case 'DG':
                    topology_exc = topology_DG_exc
                case 'CA1':
                    topology_exc = topology_CA1_exc
                case 'CA3':
                    topology_exc = topology_CA3_exc
                case 'Sub':
                    topology_exc = topology_Sub_exc

        # Add different implementation for cubical!!!!
        stimulus_mask_exc = create_spherical_mask(topology_exc, stimulus_geometry_settings)

        
        ###########################################
        ########  Creating Treatment Mask  ########
        ###########################################
        # Defining which kind of treatment mask to use.
        treatment_mask = "all_neurons"
        treatment_mask = "all_layer_EC"
        treatment_mask = "spherical_layer_EC"
        
        # Creating the treatment mask where only a sphere of neurons in the middle of the area is included.
        treatment_geometry_settings = [current_variables['coord_of_electrode'], current_variables['radius_of_electrode']]
        treatment_mask_exc = create_spherical_mask(topology_exc, treatment_geometry_settings)
        treatment_mask_inh = create_spherical_mask(topology_inh, treatment_geometry_settings)

        # # Creating the treatment mask where all the neurons are included.
        # treatment_mask_exc = np.ones(current_variables['N'][0])
        # treatment_mask_inh = np.ones(current_variables['N'][1])
        
        treatment_masks = [treatment_mask_exc, treatment_mask_inh]


        ########################################
        ########  Initializing Network  ########
        ########################################
        # Instantiating the network and setting up the monitors.
        net, synapses, monitors = prepare_network(topologies, stimulus_mask_exc, treatment_masks, current_variables)
        popmon_exc, popmon_inh, statemon_exc, statemon_inh, Mlfp = monitors
        
        # Writing the network statistics to a file.
        write_network_statistics(synapses, current_variables['N'], run_id)

        # Plotting and saving the neuron stimulus and treatment mask.
        plot_neuron_masks(topology_exc, [stimulus_mask_exc, treatment_mask_exc], run_id)
        plot_neuron_mask(topology_exc, stimulus_mask_exc, 'red', 'stimulus', run_id)
        plot_neuron_mask(topology_exc, treatment_mask_exc, 'blue', 'treatment', run_id)

        try:
            # Running the simulation with the dynamic objects created above.
            run_granular_simulation(net, current_variables, treatment_settings, monitors)

            # Saving the firing rate data.
            np.savetxt(f'./results/{run_id}/fr_exc.txt', popmon_exc.rate)
            np.savetxt(f'./results/{run_id}/fr_inh.txt', popmon_inh.rate)

            # Plotting the firing rate data together with the noise for both the excitatory and inhibitory neurons.
            plt.plot(popmon_inh.t, popmon_inh.rate, label='Inhibitory')
            plt.plot(popmon_exc.t, popmon_exc.rate, label='Excitatory')

            # Plotting and saving the firing rates.
            plt.legend()
            plt.savefig(f'./results/{run_id}/firing-rates.png', bbox_inches='tight')
            plt.close()

            # Plotting and saving the noise for the inhibitory neurons.
            plt.plot(statemon_inh.t, statemon_inh.I_noise[4]/nA, label='inh')
            plt.savefig(f'./results/{run_id}/noise_inh.png', bbox_inches='tight')
            plt.close()

            # Plotting and saving the noise for the excitatory neurons.
            plt.plot(statemon_exc.t, statemon_exc.I_noise[4]/nA, label='exc')
            plt.savefig(f'./results/{run_id}/noise_exc.png', bbox_inches='tight')
            plt.close()

            # Plotting and saving the recorded LFP voltage values in millivolts from the 'Mlfp' monitor. 
            np.savetxt(f'./results/{run_id}/LFP.txt', Mlfp.v[0]/mV)
            plot(Mlfp.t/ms, Mlfp.v[0]/mV)
            plt.savefig(f'./results/{run_id}/LFP.png', bbox_inches='tight')
            plt.close()

            
        except:
            print('Broken run.')
        
        time.sleep(1)

<br></br><br></br>

<h2 style="font-size: 40px;">Simulation Variables</h2>

In [32]:
########################################
########  Simulation Variables  ########
########################################

# Setting the number of times the simulation will be performed.
copy_times = 5

# Defining the layer densities and layer excitatory to inhibitory ratios.
layer_densities_dict = {"EC": 27400, 
                        "DG": 87100, 
                        "CA1": 7900, 
                        "CA3": 15100, 
                        "Sub": 21500}
excitatory_ratios_dict = {"EC": 0.8, 
                          "DG": 0.8, 
                          "CA1": 0.8, 
                          "CA3": 0.8, 
                          "Sub": 0.8}

# Defining the vocabulary of variables.
variables = {
    
    # Defining the simulation settings.
    "run_id": ['Results 1', 'Results 2', 'Results 3', 'Results 4', 'Results 5'],
    "duration": [4000*ms]*copy_times,
    "copy_times": copy_times,

    # Defining the total number of neurons of the model.
    "N": [17000]*copy_times,

    # Defining the bounds used in the random topology model.
    "bounds": [[0.6, 0.6, 0.6]]*copy_times,
    
    # Defining the number of neurons per mm^3 and the excitatory ratio for each of the layers.
    "layer_names": ['EC', 'DG', 'CA1', 'CA3', 'Sub']*copy_times,
    "layer_densities": [dict(layer_densities_dict)]*copy_times,
    "excitatory_ratios": [dict(excitatory_ratios_dict)]*copy_times,

    # Defining the potassium equilibrium potential for both the excitatory and inhibitory neurons.
    # - Healthy mode: Eke_baseline = -90mV
    # - Epileptic mode: Eke_baseline = -84mV
    "Eke_baseline": [-84*mV]*copy_times, 
    "Eki_baseline": [-90*mV]*copy_times,

    # Defining the noise affecting the excitatory and inhibitory neurons.
    "noise_exc": [[0.07, 0.075]*nA]*copy_times, # OLD: [0.1045, 0.104]
    "noise_inh": [[0.05, 0.08]*nA]*copy_times,

    # Defining the base probabilities of connections between neurons:
    # - p_e2e => Probability of an excitatory to excitatory neuron (synapse) connection.
    # - p_e2i => Probability of an excitatory to inhibitory neuron (synapse) connection.
    # - p_i2e => Probability of an inhibitory to excitatory neuron (synapse) connection.
    # - p_i2i => Probability of an inhibitory to inhibitory neuron (synapse) connection.
    # Normal ranges from 0.7-0.75, to activate sprouting increase the normal by 0.5
    # This will increase the average number of excitatory connections by 500.
    "p": [[0.75, 0.35, 0.35, 0.0]]*copy_times, 

    # Defining from which file the stimulus originates.
    "input_signal_file": ['sigmoid-1.0.txt']*copy_times, 

    # Defining the treatment settings.
    "device_sensitivity": [8*ms]*copy_times, # Device sensitivity - how frequently to check is firing rate is above the threshold
    "firing_rate_threshold": [5*Hz]*copy_times, 
    "Eke_treatment": [-100*mV]*copy_times,
    "Eki_treatment": [-90*mV]*copy_times,



    # SHOULD BE UPDATED!!!!
    "radius_of_electrode": [200]*copy_times,
    
    
    
    
    
    "distance_between_masks": [100]*copy_times,
}

In [33]:
populate_electrode_positions(variables)

ValueError: Diameter provided is larger than box size

<br></br><br></br>

<h2 style="font-size: 40px;">Running the Simulation</h2>

In [13]:
# Running the simulation with the variables.
run_model_loop(variables)


Radius of electrode: 200
Max distance possible between masks: 173.20508075688772
Distance selected: 100


Radius of electrode: 200
Max distance possible between masks: 173.20508075688772
Distance selected: 100


Radius of electrode: 200
Max distance possible between masks: 173.20508075688772
Distance selected: 100


Radius of electrode: 200
Max distance possible between masks: 173.20508075688772
Distance selected: 100


Radius of electrode: 200
Max distance possible between masks: 173.20508075688772
Distance selected: 100

[70. 75.] pA


cl : Command line warning D9025 : overriding '/W3' with '/w'
WARNING    Cannot use Cython, a test compilation failed: command 'C:\\Program Files (x86)\\Microsoft Visual Studio\\2022\\BuildTools\\VC\\Tools\\MSVC\\14.39.33519\\bin\\HostX86\\x64\\cl.exe' failed with exit code 2 (CompileError) [brian2.codegen.runtime.cython_rt.cython_rt.failed_compile_test]
INFO       Cannot use compiled code, falling back to the numpy code generation target. Note that this will likely be slower than using compiled code. Set the code generation to numpy manually to avoid this message:
prefs.codegen.target = "numpy" [brian2.devices.device.codegen_fallback]


_cython_magic_53cfa03f278cdb8b8876fb3be0f858b3.cpp
C:\Users\laure\.cython\brian_extensions\_cython_magic_53cfa03f278cdb8b8876fb3be0f858b3.cpp(42): fatal error C1083: Cannot open include file: 'Python.h': No such file or directory
#######################
# Starting Simulation #
#######################

Treatment Parameters: time sensitivity 8. ms FR Threshold: 5. Hz



Running Simulation:   0%|          | 0/500 [00:00<?, ?it/s]

WARNING    'glu' is an internal variable of group 'exc_group', but also exists in the run namespace with the value 1. The internal variable will be used. [brian2.groups.group.Group.resolve.resolution_conflict]
WARNING    inh_group's variable 'v' has NaN, very large values, or encountered an error in numerical integration. This is usually a sign that an unstable or invalid integration method was chosen. [brian2.groups.group.invalid_values]
WARNING    inh_group's variable 'm' has NaN, very large values, or encountered an error in numerical integration. This is usually a sign that an unstable or invalid integration method was chosen. [brian2.groups.group.invalid_values]
WARNING    inh_group's variable 'h' has NaN, very large values, or encountered an error in numerical integration. This is usually a sign that an unstable or invalid integration method was chosen. [brian2.groups.group.invalid_values]
WARNING    inh_group's variable 'n' has NaN, very large values, or encountered an error in 

...

Running Simulation:   0%|          | 0/500 [00:00<?, ?it/s]

[70. 75.] pA
#######################
# Starting Simulation #
#######################

Treatment Parameters: time sensitivity 8. ms FR Threshold: 5. Hz



Running Simulation:   0%|          | 0/500 [00:00<?, ?it/s]

[70. 75.] pA
#######################
# Starting Simulation #
#######################

Treatment Parameters: time sensitivity 8. ms FR Threshold: 5. Hz



Running Simulation:   0%|          | 0/500 [00:00<?, ?it/s]